In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.metrics import mean_absolute_error as mae


# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()


# For time stamps
from datetime import datetime

In [ ]:
start_date = datetime(2015,1,1)
end_date = datetime.now()


STC = pdr.get_data_yahoo('7010.SR', start_date, end_date)
Aramco = pdr.get_data_yahoo('2222.SR', start_date, end_date)
Rajhi = pdr.get_data_yahoo('1120.SR', start_date, end_date)
Jarir = pdr.get_data_yahoo('4190.SR', start_date, end_date)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
df = pdr.get_data_yahoo('1120.SR', start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-01,21.226557,21.328117,20.921867,21.023432,14.864165,6688117
2015-01-04,20.921867,21.429682,20.718742,21.226557,15.007778,8714557
2015-01-05,20.921867,21.124992,20.718742,20.820307,14.720550,6415201
2015-01-06,20.414057,21.023432,20.231243,20.921867,14.792352,10596374
2015-01-07,20.820307,21.124992,20.515617,20.820307,14.720550,17053203
...,...,...,...,...,...,...
2023-08-15,72.599998,72.699997,72.199997,72.599998,72.599998,2198402
2023-08-16,72.099998,72.900002,71.599998,72.300003,72.300003,3426989
2023-08-17,72.099998,73.199997,71.900002,73.000000,73.000000,3995546


In [ ]:
fig = px.line(df, x=df.index, y='Close', title='Close Price History')
fig.show()


In [ ]:
# Create a new dataframe with only the 'Close column
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .90 ))

training_data_len

1973

In [ ]:
# # Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(dataset)

In [ ]:
scaled_data.shape

(2192, 1)

In [ ]:
dataset.shape

(2192, 1)

In [ ]:
train_data = dataset[0:int(training_data_len), :]


In [ ]:
train_data.shape

(1973, 1)

In [ ]:
train_data.shape


(1973, 1)

In [ ]:
# Create the training data set
# Create the scaled training data set

# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(scaler.fit_transform(train_data[i-60:i, 0].reshape(-1,1)))
    y_train.append(train_data[i, 0])


# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(scaler.fit_transform(test_data[i-60:i, 0]).reshape(-1,1))

# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

In [ ]:
x_train.shape

(1913, 60, 1)

In [ ]:
x_test.shape

(219, 60, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=False))

model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(x_train, y_train, batch_size=16, epochs=10)
# history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, shuffle=True)


Epoch 1/10
 84/120 [====================>.........] - ETA: 15s - loss: 18.8720

KeyboardInterrupt: ignored

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(rmse)


MAE = mae(y_test, predictions)
print(MAE)

7/7 [==============================] - 1s 42ms/step
1.3681675455505324
1.0028535677417774


In [ ]:
# Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions


<ipython-input-82-c6cb20b0ec05>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = px.line()

# Adding the train data
fig.add_scatter(x=train.index, y=train['Close'], mode='lines', name='Train')

# Adding the valid data and predictions
fig.add_scatter(x=valid.index, y=valid['Close'], mode='lines', name='Validation')
fig.add_scatter(x=valid.index, y=valid['Predictions'], mode='lines', name='Predictions')

# Customizing the plot
fig.update_layout(
    title='Model',
    xaxis_title='Date',
    yaxis_title='Close Price',
)

fig.show()

In [ ]:
# Show the valid and predicted prices
valid['Difference'] = valid['Close'] - valid['Predictions']

<ipython-input-84-acbb63dcddf7>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
valid

,Close,Predictions,Difference
Date,,,
2022-09-29,81.300003,78.246063,3.053940
2022-10-02,82.500000,78.858360,3.641640
2022-10-03,83.000000,80.098000,2.902000
2022-10-04,83.000000,81.374649,1.625351
2022-10-05,84.800003,82.297836,2.502167
...,...,...,...
2023-08-13,72.699997,71.095779,1.604218
2023-08-14,72.599998,71.806671,0.793327
2023-08-15,72.599998,72.320908,0.279091
